In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Add, GlobalAveragePooling2D, Dense, Flatten, Conv2D, Lambda, Input, BatchNormalization, Activation
from tensorflow.keras.optimizers import schedules, SGD
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
# def load_dataset():
    """
    Load the FER2013 dataset.
    """
file_path = '/kaggle/input/fer2013/fer2013.csv'
data = pd.read_csv(file_path)

pixels = data['pixels'].tolist()
width, height = 48, 48
faces = []
for pixel_sequence in pixels:
    face = np.asarray([int(pixel) for pixel in pixel_sequence.split(' ')]).reshape(width, height)
    face = np.stack((face,)*3, axis=-1)  # Convert to RGB
    faces.append(face)
faces = np.array(faces)
faces = faces.astype('float32') / 255.0

emotions = pd.get_dummies(data['emotion']).values

# Split the data into training and testing sets
train_size = int(0.8 * len(faces))
input_train, input_test = faces[:train_size], faces[train_size:]
target_train, target_test = emotions[:train_size], emotions[train_size:]

#     return (X_train, y_train), (X_test, y_test)

# (input_train, target_train), (input_test, target_test) = (X_train, y_train), (X_test, y_test)

input_train = X_train
target_train = y_train
input_test = X_test
target_test = y_test
# print("",input_train.shape)

width, height, dim = 48, 48, 3
num_classes = 7
# def preprocess_image(image):
#     image = tf.image.resize(image, (48, 48))
#     return image

# Data augmentation: perform zero padding on datasets
# paddings = tf.constant([[0, 0], [4, 4], [4, 4], [0, 0]])
# input_train = tf.pad(input_train, paddings, mode="CONSTANT")

# Data generator for training data with additional augmentations
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=10,          # Random rotations
    width_shift_range=0.1,      # Random horizontal shifts
    height_shift_range=0.1,     # Random vertical shifts
    zoom_range=0.1,             # Random zoom
    shear_range=0.1,            # Random shear
    rescale=1./255
#     preprocessing_function=preprocess_image
)

# Generate training and validation batches
train_batches = train_generator.flow(input_train, target_train, batch_size=32, subset="training")
validation_batches = train_generator.flow(input_train, target_train, batch_size=32, subset="validation")
# batch_x, batch_y = next(train_batches)
# print(batch_x.shape)
# Data generator for testing data
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

# Generate test batches
test_batches = test_generator.flow(input_test, target_test, batch_size=32)